In [ ]:
!pip install transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 58.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# BERT 모델과 토크나이저 로드
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# 주제를 추출하고자 하는 문장 또는 문서 입력
text = "This is a sample text about natural language processing and machine learning."

# 입력 문장을 토큰화하고 BERT 모델에 입력할 형식으로 변환
input_ids = tokenizer.encode(text, add_special_tokens=True, max_length=512, truncation=True, padding=True, return_tensors="pt")

# 주제 추출을 위한 모델 예측
with torch.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits

# 주제 클래스(카테고리) 중에서 가장 확률이 높은 것 선택
predicted_class = torch.argmax(logits, dim=1).item()

# 예측된 주제를 출력
print(f"Predicted Topic Class: {predicted_class}")


## 유사도 분석
- TFIDF, 코사인 유사도 활용

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

# 입력 텍스트 데이터
texts = ["맛이 정말 좋아요. 맛있어요.",
         "양도 푸짐하고 좋아요.",
         "가격 대비 맛있어요.",
         "서비스가 빠르고 좋아요.",
         "맛과 가격이 모두 좋아요."]

# 키워드
keywords = ["맛", "양", "가격"]

# TF-IDF 벡터 생성
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

# 키워드 벡터 생성
keyword_vectors = vectorizer.transform(keywords)

# 코사인 유사도 계산
similarities = cosine_similarity(keyword_vectors, tfidf_matrix)

# 각 키워드와 관련된 문장 추출
for i, keyword in enumerate(keywords):
    related_sentences = [texts[j] for j in similarities[i].argsort()[::-1]]
    print(f"{keyword}와(과) 관련된 문장:")
    for sentence in related_sentences:
        print("-", sentence)


맛와(과) 관련된 문장:
- 맛과 가격이 모두 좋아요.
- 서비스가 빠르고 좋아요.
- 가격 대비 맛있어요.
- 양도 푸짐하고 좋아요.
- 맛이 정말 좋아요. 맛있어요.
양와(과) 관련된 문장:
- 맛과 가격이 모두 좋아요.
- 서비스가 빠르고 좋아요.
- 가격 대비 맛있어요.
- 양도 푸짐하고 좋아요.
- 맛이 정말 좋아요. 맛있어요.
가격와(과) 관련된 문장:
- 가격 대비 맛있어요.
- 맛과 가격이 모두 좋아요.
- 서비스가 빠르고 좋아요.
- 양도 푸짐하고 좋아요.
- 맛이 정말 좋아요. 맛있어요.


## 토픽 매칭 사전학습 모델


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("amberoad/bert-multilingual-passage-reranking-msmarco")

model = AutoModelForSequenceClassification.from_pretrained("amberoad/bert-multilingual-passage-reranking-msmarco")


In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# 모델과 토크나이저 로드
model_name = "amberoad/bert-multilingual-passage-reranking-msmarco"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 리뷰 데이터
reviews = [
    "맛있어요. 정말 맛있어요.",
    "양이 푸짐하고 좋아요.",
    "가격 대비 맛있어요.",
    "서비스가 빠르고 좋아요.",
    "맛과 가격이 모두 좋아요."
]

# 주제별 쿼리 설정
queries = ["맛", "양", "가격", "서비스"]

# 주제별 관련성 점수를 저장할 리스트
scores_by_topic = []

# 각 주제별로 모델 예측 수행
for query in queries:
    # 리뷰와 주제별 쿼리를 토큰화
    inputs = tokenizer(reviews, query, return_tensors="pt", padding=True, truncation=True, max_length=128)

    # 모델 예측
    with torch.no_grad():
        logits = model(**inputs).logits

    # 관련성 점수 추출
    scores = logits[:, 1].tolist()  # 레이블 1에 대한 점수를 추출 (관련성 점수)
    scores_by_topic.append(scores)

# 주제별 관련성 점수 출력
for i, query in enumerate(queries):
    print(f"주제 '{query}'에 대한 관련성 점수:")
    for j, review in enumerate(reviews):
        print(f"- 리뷰 {j + 1}: {scores_by_topic[i][j]}")


TypeError: ignored

In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# 모델과 토크나이저 로드
model_name = "amberoad/bert-multilingual-passage-reranking-msmarco"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# 리뷰 데이터
reviews = [
    "맛있어요. 정말 맛있어요.",
    "양이 푸짐하고 좋아요.",
    "가격 대비 맛있어요.",
    "서비스가 빠르고 좋아요.",
    "맛과 가격이 모두 좋아요."
]

# 주제별 쿼리 설정
queries = ["맛", "양", "가격", "서비스"]

# 주제별 관련성 점수를 저장할 리스트
scores_by_topic = []

# 각 주제별로 모델 예측 수행
for query in queries:
    # 리뷰와 주제별 쿼리를 토큰화
    inputs = tokenizer(reviews, [query] * len(reviews), return_tensors="pt", padding=True, truncation=True, max_length=128)

    # 모델 예측
    with torch.no_grad():
        logits = model(**inputs).logits

    # 관련성 점수 추출
    scores = logits[:, 1].tolist()  # 레이블 1에 대한 점수를 추출 (관련성 점수)
    scores_by_topic.append(scores)

# 주제별 관련성 점수 출력
for i, query in enumerate(queries):
    print(f"주제 '{query}'에 대한 관련성 점수:")
    for j, review in enumerate(reviews):
        print(f"- {review}: {scores_by_topic[i][j]}")



주제 '맛'에 대한 관련성 점수:
- 맛있어요. 정말 맛있어요.: 0.04013018310070038
- 양이 푸짐하고 좋아요.: -2.338395357131958
- 가격 대비 맛있어요.: -2.6419355869293213
- 서비스가 빠르고 좋아요.: -2.5020735263824463
- 맛과 가격이 모두 좋아요.: -2.3940234184265137
주제 '양'에 대한 관련성 점수:
- 맛있어요. 정말 맛있어요.: -2.4318349361419678
- 양이 푸짐하고 좋아요.: -1.7715847492218018
- 가격 대비 맛있어요.: -2.6851212978363037
- 서비스가 빠르고 좋아요.: -2.7551934719085693
- 맛과 가격이 모두 좋아요.: -2.464407205581665
주제 '가격'에 대한 관련성 점수:
- 맛있어요. 정말 맛있어요.: -2.8548901081085205
- 양이 푸짐하고 좋아요.: -2.373436689376831
- 가격 대비 맛있어요.: -2.3411905765533447
- 서비스가 빠르고 좋아요.: -2.6248505115509033
- 맛과 가격이 모두 좋아요.: -2.060382843017578
주제 '서비스'에 대한 관련성 점수:
- 맛있어요. 정말 맛있어요.: -2.8132286071777344
- 양이 푸짐하고 좋아요.: -3.001352548599243
- 가격 대비 맛있어요.: -2.8289682865142822
- 서비스가 빠르고 좋아요.: -2.074077844619751
- 맛과 가격이 모두 좋아요.: -3.0835487842559814


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
import pandas as pd
import numpy as np
import re
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/생성 AI 모델링/data/"
# DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cpu'

In [13]:
yogiyo =pd.read_csv(f"{DATA_PATH}yogiyo_reviews_jsi_all.csv")


In [14]:
train= yogiyo[['고객리뷰','사장댓글']]

rename_cols = ['review', 'reply']
train.columns = rename_cols
train.head()

,review,reply
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [15]:
review_list = []
for review in train['review']:
    if isinstance(review, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", review)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        review_list.append(result)
    else:
        review_list.append("")

train['review'] = review_list
train.head()

<ipython-input-15-6374a3c305ac>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['review'] = review_list


,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...


In [16]:
reply_list = []
for reply in train['reply']:
    if isinstance(reply, str):

        pat = re.compile("[^a-zA-Z가-힣0-9.,~?!]")
        result = pat.sub(" ", reply)

        result = re.sub(r'\.{2,}', '.', result)
        result = re.sub(r'\,{2,}', ',', result)
        result = re.sub(r'\~{2,}', '~', result)
        result = re.sub(r'\!{2,}', '!', result)
        result = re.sub(r'\?{2,}', '?', result)
        result = re.sub(" +", " ", result)
        reply_list.append(result)
    else:
        reply_list.append("")

train['reply'] = reply_list
train.head()

<ipython-input-16-2764c488e75c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['reply'] = reply_list


,review,reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...


In [18]:
train['review']

0         사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~
1                           리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!
2                       잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.
3                                       굿 말이 필요 없는 메뉴 였습니다
4                                              맛있어요 양도많구좋음
                               ...                        
18681                                          맛있게 잘 먹었어요 
18682            요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다
18683                                 배달 빨라요 맛있게 잘 먹었습니당 !
18684                                              잘 먹었어요~
18685     식후감 자주 시키는데 항상 요청사항도 잘들어주시고 맛있고 빨라요 앞으로도 자주 시킬게요
Name: review, Length: 18686, dtype: object